<a href="https://colab.research.google.com/github/wetherc/data-3000/blob/main/labs/02_20_neural_network_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ucimlrepo

In [5]:
import pandas as pd
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

# metadata
print(cdc_diabetes_health_indicators.metadata)

# variable information
print(cdc_diabetes_health_indicators.variables)

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

In [6]:
import pandas as pd

X = pd.DataFrame(X)

int_cols = [
    'BMI',
    'Education',
    'Income',
    'Age',
    'GenHlth',
    'MentHlth',
    'PhysHlth'
]

In [7]:
# Compute Z-scores for each numeric column in our dataset
# and rescale the original values

for col in int_cols:
    X[col] = (X[col] - X[col].mean()) / X[col].std()

In [8]:
X.loc[:, int_cols].describe()

,BMI,Education,Income,Age,GenHlth,MentHlth,PhysHlth
count,2.536800e+05,2.536800e+05,2.536800e+05,2.536800e+05,2.536800e+05,2.536800e+05,2.536800e+05
mean,-2.455865e-16,-1.281711e-16,6.453368e-17,9.500792e-17,2.097345e-16,-2.083900e-17,8.828566e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.478911e+00,-4.108886e+00,-2.440133e+00,-2.302427e+00,-1.414529e+00,-4.296288e-01,-4.865915e-01
25%,-6.631210e-01,-1.065593e+00,-5.088362e-01,-6.653479e-01,-4.786178e-01,-4.296288e-01,-4.865915e-01
50%,-2.091735e-01,-5.116143e-02,4.568120e-01,-1.051634e-02,-4.786178e-01,-4.296288e-01,-4.865915e-01
75%,3.960898e-01,9.632697e-01,9.396361e-01,6.443152e-01,4.572934e-01,-1.598269e-01,-1.424739e-01
max,1.053425e+01,9.632697e-01,9.396361e-01,1.626563e+00,2.329116e+00,3.617399e+00,2.954584e+00


In [23]:
X['diet'] = X['Fruits'] + X['Veggies'] - X['HvyAlcoholConsump']

In [24]:
X['diet'].describe()

count    253680.000000
mean          1.389479
std           0.733420
min          -1.000000
25%           1.000000
50%           2.000000
75%           2.000000
max           2.000000
Name: diet, dtype: float64

In [27]:
X.drop(['Age'], axis=1, inplace=True)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    random_state=42,
    train_size=0.7)

In [31]:
X_train.shape

(177576, 21)

In [32]:
import tensorflow as tf

In [33]:
inputs = {
    'GenHlth':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='GenHlth'),
    'BMI':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='BMI'),
    'PhysHlth':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='PhysHlth'),
    'diet':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='Diet'),
    'HighBP':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='HighBP'),
    'HighChol':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='HighChol'),
}

In [ ]:
preprocessing_layers = tf.keras.layers.Concatenate()(
    [inputs.get('GenHlth'), inputs.get('BMI'),  inputs.get('PhysHlth'),
     inputs.get('diet'),  inputs.get('HighBP'),  inputs.get('BMHighChol')])

hidden = tf.keras.layers.Dense(1000, activation='sigmoid')(preprocessing_layers)
hidden = tf.keras.layers.Dense(500, activation='sigmoid')(hidden)

output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)